In [64]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
import src.annotation_utils as a_utils
import src.llm_utils as llm_utils
import src.message_utils as m_utils
from src.env import (
    BRAT_DATA_PATH,
)
from src import utils
import os

load_dotenv()


True

## Load brat dataset

In [65]:

import dataclasses

from pybrat.parser import BratParser, Entity, Event, Example, Relation

# Initialize a parser.
brat = BratParser(error="ignore")
# brat = BratParser()
annotations = brat.parse(BRAT_DATA_PATH)


In [66]:
# data_entities = a_utils.get_data_entities_of_segments(annotations)
data_entities = a_utils.get_data_entities_of_sentences(annotations)

len(data_entities), data_entities[10]

(1067,
 {'segment': 'INTRODUCTION\n\nDomains Holdings Group Limited (the "Company"), offers a global network of content-rich web, mobile and social sites, including user acquisition and media buying (including paid search, display, mobile and social advertising), content optimization (including key word search, off-page and on-page optimization), and affiliate program, both in the web and mobile ecosystem (including through websites or mobile apps operated by Company; collectively, the "Services").\nThe Company is deeply committed to safeguard the privacy expectations of its end users ("User(s)", "you" or "your"). Accordingly, we have put in place this Privacy Policy, which outlines our data protection practices, including how we collect, use, disclose and protect your Personal Information, as well as your rights with respect to your Personal Information.\nIn this Privacy Policy, you can learn about:\nWHAT INFORMATION WE COLLECT?\nMINORS\nHOW WE USE THE COLLECTED INFORMATION?\nMARKETIN

## Select training data

In [67]:
def has_no_entities(segment):
    return not segment["entities"]

NUM_TRAIN_HAS_ENTITY = 20
NUM_TRAIN_NO_ENTITY = 10
NUM_VAL_HAS_ENTITY = 3
NUM_VAL_NO_ENTITY = 2

empty_entity_indices = [i for i, segment in enumerate(data_entities) if has_no_entities(segment)]

training_set_indices = []
validation_set_indices = []
for i, segment in enumerate(data_entities):
    if not has_no_entities(segment):
        if len(training_set_indices) < 20:
            training_set_indices.append(i)
        elif len(validation_set_indices) < 3:
            validation_set_indices.append(i)

for i in empty_entity_indices:
    if len(training_set_indices) < 30:
        training_set_indices.append(i)
    elif len(validation_set_indices) < 5:
        validation_set_indices.append(i)

len(training_set_indices), len(validation_set_indices)

(30, 5)

In [68]:
def has_no_entities(segment):
    return not segment["entities"]

NUM_TRAIN_HAS_ENTITY = 10
NUM_TRAIN_NO_ENTITY = 30
NUM_VAL_HAS_ENTITY = 2
NUM_VAL_NO_ENTITY = 6

empty_entity_indices = [i for i, segment in enumerate(data_entities) if has_no_entities(segment)]
non_empty_entity_indices = [i for i, segment in enumerate(data_entities) if not has_no_entities(segment)]

training_set_indices = []
validation_set_indices = []

indices, idx = utils.evenly_get(empty_entity_indices, NUM_TRAIN_NO_ENTITY)
training_set_indices += indices.tolist()
empty_entity_indices_remaining = np.delete(empty_entity_indices, idx)
indices, idx = utils.evenly_get(empty_entity_indices_remaining, NUM_VAL_NO_ENTITY)
validation_set_indices += indices.tolist()
indices, idx = utils.evenly_get(non_empty_entity_indices, NUM_TRAIN_HAS_ENTITY)
training_set_indices += indices.tolist()
non_empty_entity_indices_remaining = np.delete(non_empty_entity_indices, idx)
indices, idx = utils.evenly_get(non_empty_entity_indices_remaining, NUM_VAL_HAS_ENTITY)
validation_set_indices += indices.tolist()

len(empty_entity_indices), len(non_empty_entity_indices), len(training_set_indices), len(validation_set_indices)

(973, 94, 40, 8)

## Fine-tune model with specified training data

In [69]:
# all_data = m_utils.as_training_data_for_data_span_of_segment(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_segment_1_1(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence_only(data_entities)
all_data = m_utils.as_training_data_for_data_classification_of_segment(data_entities)
# all_data = m_utils.as_training_data_for_data_classification_of_segment_gradual(data_entities)

base_model = 'gpt-4o-2024-08-06'
# base_model = 'gpt-4o-mini-2024-07-18'
suffix = 'data_class-seg_sent_data-v2'
# suffix = 'sent-data-entities-30-train-5-val-with-empty'

In [61]:
job_desc_dir, fine_tune_job, test_set = llm_utils.fine_tune_with_data(
    all_data, training_set_indices, validation_set_indices,
    basemodel=base_model,
    fine_tune_args={'suffix':suffix},
    desc='data_class-seg_sent_data-v2')

job_desc_dir, fine_tune_job, len(test_set), test_set[0]

Constructing training and validation data files (locally)...
Uploading training and validation data files...
Data files uploading (API calls returned).
Creating fine-tuning job...
Created fine-tuning job.
Job description saved.


(PosixPath('../out/fine_tune-2024-09-18-19-28-00-gpt-4o-2024-08-06'),
 FineTuningJob(id='ftjob-hnXjy0dmzty3pzHdsXLOY57u', created_at=1726684083, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=[], seed=797148961, status='validating_files', trained_tokens=None, training_file='file-22aNMxBBxYsNwEOWhcEY9XIE', validation_file='file-XJDj4r7TbLRc4sNmBTgh5Y2t', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='renyuneyun-university-of-oxford', entity=None, name=None, tags=None, run_id='ftjob-hnXjy0dmzty3pzHdsXLOY57u'))], user_provided_suffix='data_entity-seg_sent_data-v2'),
 1019,
 {'messages': [{'role': 'system',
    'content': 'You are an annotation expert

In [14]:
%debug

> /home/ryey/workspaces/oxford/PP-DToU/fine-tune/.venv/lib/python3.12/site-packages/openai/_base_client.py(1041)_request()
   1039 
   1040             log.debug("Re-raising status error")
-> 1041             raise self._make_status_error_from_response(err.response) from None
   1042 
   1043         return self._process_response(

   1036             # to completion before attempting to access the response text.
   1037             if not err.response.is_closed:
   1038                 err.response.read()
   1039 
   1040             log.debug("Re-raising status error")
-> 1041             raise self._make_status_error_from_response(err.response) from None
   1042 
   1043         return self._process_response(
   1044             cast_to=cast_to,
   1045             options=options,
   1046             response=response,

   1047             stream=stream,
   1048             stream_cls=stream_cls,
   1049             retries_taken=options.get_max_retries(self.max_retries) - retries,

In [41]:
llm_utils.await_fine_tune_finish_and_clean_up(wait_for_job_completion=False)

FineTuningJob(id='ftjob-61c0FVgYkOfjZaMwRpOjNOzo', created_at=1726265855, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:rui:30-train-5-val-with-empty-from-api:A790JXld', finished_at=1726266269, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=['file-1TTTIJ6t7DVXg51yYafkwd4J'], seed=1504396448, status='succeeded', trained_tokens=68019, training_file='file-iYtsamFpHaR3FyqGhHWNeMVm', validation_file='file-C1S6NC5e9VWZrwgmuFzkDxVR', estimated_finish=None, integrations=[], user_provided_suffix='30_train-5_val-with_empty-from_api')

In [31]:
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30_train-5_val-content_only-from_api:A6KhgqGM'
fine_tuned_model_id = 'ft:gpt-4o-mini-2024-07-18:rui:30-train-5-val-content-only-from-api:A6cig7w6'